In [88]:
#verificar com o ayres [[]] aninhado

import json
import re

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [90]:
#print(data[0].keys())

In [92]:
#print(data[0]['title'])

In [94]:
#print(data[0]['body'])

---

Como vocês podem ver, o texto está cheio de caracteres de estruturação de documento.

---

**Atividade:** Consulte a documentação da Wikipedia para descobrir o que são as marcações `[[`, `]]`, `{{` e ```}}```

**R:** O que está entre [[ ]] é um hypertexto. o que está entre {{}} é um supertexto (ex: aaaa², o 2 é um supertexto)

---

Vamos explorar o corpus para minerar itens de interesse.

**Exercicio:** Faça uma função que retorna uma lista com todos os links html de um texto deste corpus.

In [96]:
# def find_links(text):
#     return re.findall(r'\[\[.+?\]\]', text)
# find_links(" evoluir.thumb|esquerda|300px|Primeira edi\u00e7\u00e3o do Unua Libro, em [[L\u00edngua russa|russo]]O primeiro livro sobre o esperanto foi")

In [98]:
#print(find_links(data[2]['body']))

**Exercicio:** Faça uma função que recebe um texto do corpus e substitui todas as ocorrências de links da Wikipedia (textos entre os tags `[[` e `]]`) por texto simples, e retorna o documento limpo. Por exemplo, quando encontrar algo como `[[Etimologia|Etimologicamente]]` substituir por `Etimologicamente`. Você pode supor que os links não são *aninháveis*, como por exemplo `[[blablabla[[etc]]blebleble]]`.

In [100]:
def replace_links(texto):
    pattern = r'''
        \[\[
        (?:
            [^|]*?\|
        )*?
        (
            [^|]*?
        )
        \]\]
    '''
    replacement = r'\1' # primeiro grupo de captura
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(replacement, texto)

**Exercicio**: Faça uma função que recebe um texto do corpus e remove todas as ocorrências de referências (textos entre os tags `<ref>` e `</ref>`).

In [102]:
def remove_ref(text):
    return re.sub(r'<ref(?:.*)?>.*?(?:</ref>)?', '', text)

In [104]:
def remove_math(text):
    return re.sub(r'<math(?:.*)?>.*?(?:</math>)?', '', text)

In [106]:
def remove_small(text):
    return re.sub(r'<.*?>(.*?)<.*?>', r'\1', text)

In [108]:
#replace vss sub
def limpa_aspas(texto):
    pattern = r"""(['"]+)(.*?)\1"""
    matcher = re.compile(pattern)
    return matcher.sub(r'\2', texto)

In [110]:
def remove_equal(text):
    return text.replace('=',' ') 

In [112]:
#Remove {}
def limpa_chaves(texto):
    pattern = r"{.*?}"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)


In [114]:
def limpa_colchete(texto):
    pattern = r"\[.*?\]"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)

In [116]:
def limpa_obs(texto):
    pattern = r"<!--.*?-->"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)

In [118]:
def remove_ast(texto):
    texto = texto.replace("*", "")
    return texto

In [120]:
def remove_px(text):     
    return re.sub(r"(?:\d+x)?\d+px", ' ', text)

remove_px("o problema.624pxDurante a adolescênci")

'o problema. Durante a adolescênci'

In [122]:
def remove_dash(text):
    return re.sub(r'\s+[-–]\s+', ' ', text)

remove_dash("Vista - Alegre Sport Clube Beira-Mar")

'Vista Alegre Sport Clube Beira-Mar'

In [124]:
def remove_remaining_brackets(text):
    text = text.replace(']', '')
    return text.replace('[', '')

In [126]:
def remove_parenthesis(text):
    text = text.replace('(', '')
    return text.replace(')', '')

In [128]:
def ignore_digits(text):
    return re.sub('\d+', '', text)

**Exercicio:** Faça uma função que recebe um texto do corpus e remove todos os templates (textos entre os tags `{{` e `}}`). Este exercício é desafiante: *os templates podem ser aninhados*! E agora, como proceder?

In [130]:
def remove_templates(text):
    conta = 0
    spans_proibidos = []
    for item in re.finditer(r'{{|}}', text):
        if item[0] == '{{':
            if conta == 0:
                inicio = item.span()[0]
            conta += 1
        else:
            conta -= 1
            if conta == 0:
                fim = item.span()[1]
                spans_proibidos.append((inicio, fim))

    clean_text = ''
    inicio = 0
    for span in spans_proibidos:
        fim, novo_inicio = span
        clean_text += text[inicio:fim]
        inicio = novo_inicio
    
    clean_text += text[inicio:]
    return clean_text



**Exercicio:** Usando os codigos desenvolvidos acima, faça uma função que recebe um texto do corpus e retorna a sua versão limpa.

In [132]:
def clear_text(text):
    funcs = [
        replace_links,
        limpa_aspas,
        remove_ref,
        remove_equal,
        remove_templates,
        remove_math,
        remove_small,
        limpa_colchete,
        remove_ast,
        remove_px,
        remove_dash,
        remove_remaining_brackets,
        remove_parenthesis,
        ignore_digits,
        limpa_chaves
    ]
    x = text
    for func in funcs:
        x = func(x)
    return x

**Exercicio:** Limpe todos os documentos e explore os resultados para ver o que mais dá para limpar. Nosso objetivo é ter uma coletânea de textos limpos para poder criar um vocabulário da língua portuguesa!

In [134]:
import json
with open('dump_small_clean.jsonln', 'w', encoding="utf8") as file:
    # for i in range(len(data)):
    for i in range(10):
        
        json.dump({"body": clear_text(data[i]['body']), "title": data[i]["title"]}, file)
        file.write('\n')

# Criando um tokenizador

In [136]:
# import nltk
# nltk.download('punkt')

In [138]:
data_clean = []
with open('dump_small_clean.jsonln', 'r', encoding="utf8") as file:
    for line in file:
        data_clean.append(json.loads(line))
        

In [140]:
from nltk import word_tokenize

texto = clear_text(data[0]['body'])

tokens = word_tokenize(texto)
# tokens

In [142]:
def clean_tokens(tokens):
    return [w for w in tokens if re.fullmatch(r'\w+', w)]

In [144]:
tokens = clean_tokens(tokens)

In [146]:
from tqdm import tqdm
all_words = []
for item in tqdm(data_clean):
    texto = item['body']
    tokens = word_tokenize(texto)
    tokens = clean_tokens(tokens)
    all_words += tokens


100%|██████████| 10/10 [00:00<00:00, 29.29it/s]
